# Pagination Examples

This notebook demonstrates various pagination patterns and best practices for working with large datasets in the Neon CRM API. Proper pagination is crucial for:

- **Performance**: Handling large datasets efficiently
- **Memory Management**: Avoiding loading too much data at once
- **API Limits**: Respecting rate limits and response size limits
- **User Experience**: Providing responsive applications

## What You'll Learn

- Basic automatic pagination with the SDK
- Manual pagination control for specific use cases
- Pagination with search requests
- Filtering combined with pagination
- Bulk processing strategies
- Performance optimization tips

## Prerequisites

Before running this notebook, make sure you have:
1. Installed the Neon CRM SDK: `pip install neon-crm`
2. Set up your environment variables:
   - `NEON_ORG_ID`: Your organization ID
   - `NEON_API_KEY`: Your API key
3. Or update the client initialization below with your credentials

## Import Required Libraries

In [ ]:
import os
import time

from neon_crm import NeonClient, UserType
from neon_crm.types import SearchRequest

## Initialize the Neon CRM Client

In [ ]:
# Initialize the client
client = NeonClient(
    org_id=os.getenv("NEON_ORG_ID"),
    api_key=os.getenv("NEON_API_KEY"),
    environment="production",  # or "trial"
)

# Alternative: Set credentials directly (not recommended for production)
# client = NeonClient(
#     org_id="your_org_id_here",
#     api_key="your_api_key_here",
#     environment="production"
# )

print("🚀 Neon CRM client initialized for pagination examples!")

## Example 1: Basic Automatic Pagination

The SDK's `.list()` method automatically handles pagination behind the scenes, yielding items one by one across all pages.

In [ ]:
def basic_pagination_example():
    """Basic pagination using the list method."""
    print("📄 Basic Automatic Pagination")
    print("=" * 35)
    print("The SDK automatically handles pagination - you just iterate!")
    print()

    try:
        count = 0
        page_count = 1
        items_in_current_page = 0
        page_size = 5  # Small page size for demonstration

        print(f"📋 Processing accounts with page size {page_size}:")
        print()

        for account in client.accounts.list(
            page_size=page_size, user_type=UserType.INDIVIDUAL
        ):
            count += 1
            items_in_current_page += 1

            first_name = account.get("firstName", "")
            last_name = account.get("lastName", "")
            email = account.get("email", "No email")

            print(f"  {count:2}. {first_name} {last_name} ({email})")

            # Simulate page boundaries for demonstration
            if items_in_current_page >= page_size:
                print(
                    f"     └─ End of Page {page_count} ({items_in_current_page} items)"
                )
                print()
                page_count += 1
                items_in_current_page = 0

            # Limit total results for demo
            if count >= 15:
                print(f"  ... (stopped at {count} items for demo)")
                break

        print("\n✅ Pagination Summary:")
        print(f"   Total items processed: {count}")
        print(f"   Estimated pages: {(count + page_size - 1) // page_size}")
        print(f"   Page size: {page_size}")

        print("\n💡 Key Points:")
        print("   • The SDK handles all pagination automatically")
        print("   • You just iterate through results normally")
        print("   • Memory usage stays low (streaming results)")
        print("   • No need to track page numbers manually")

    except Exception as e:
        print(f"❌ Error in basic pagination: {e}")


# Run the example
basic_pagination_example()

## Example 2: Manual Pagination Control

Sometimes you need more control over pagination - for example, to process specific pages or implement custom progress tracking.

In [ ]:
def manual_pagination_example():
    """Manual pagination control for specific use cases."""
    print("\n🎛️ Manual Pagination Control")
    print("=" * 35)
    print("Sometimes you need manual control over which pages to process.")
    print()

    current_page = 0
    page_size = 8
    total_processed = 0
    max_pages = 3  # Limit for demo

    try:
        print(f"📊 Processing first {max_pages} pages (page size: {page_size})")
        print()

        while current_page < max_pages:
            print(f"📄 Processing page {current_page + 1}...")

            # Collect one page of results
            page_items = []
            items_this_page = 0

            # Use the list method with specific page parameters
            for account in client.accounts.list(
                current_page=current_page,
                page_size=page_size,
                user_type=UserType.INDIVIDUAL,
            ):
                page_items.append(account)
                items_this_page += 1

                # Break after getting the expected page size
                if items_this_page >= page_size:
                    break

            if not page_items:
                print("   No more results - stopping")
                break

            # Process the page
            print(f"   Found {len(page_items)} items on this page:")
            for i, account in enumerate(page_items, 1):
                total_processed += 1
                account_type = account.get("userType", "Unknown")

                if account_type == "INDIVIDUAL":
                    name = (
                        f"{account.get('firstName', '')} {account.get('lastName', '')}"
                    )
                else:
                    name = account.get("companyName", "Unknown Company")

                email = account.get("email", "No email")
                print(f"     {i:2}. {name} ({email})")

            print(f"   ✅ Page {current_page + 1} completed - {len(page_items)} items")
            print()

            current_page += 1

        print("📊 Manual Pagination Summary:")
        print(f"   Pages processed: {current_page}")
        print(f"   Total items: {total_processed}")
        print(f"   Average per page: {total_processed / current_page:.1f}")

        print("\n💡 Use Cases for Manual Pagination:")
        print("   • Processing specific page ranges")
        print("   • Implementing custom progress tracking")
        print("   • Parallel processing of different pages")
        print("   • Resuming from a specific page after interruption")

    except Exception as e:
        print(f"❌ Error in manual pagination: {e}")


# Run the example
manual_pagination_example()

## Example 3: Pagination with Search Requests

Search requests also support pagination, allowing you to page through filtered results.

In [ ]:
def search_pagination_example():
    """Pagination with search requests."""
    print("\n🔍 Search with Pagination")
    print("=" * 30)
    print("Paginate through search results with custom criteria.")
    print()

    # Search with custom pagination settings
    search_request: SearchRequest = {
        "searchFields": [
            {"field": "Account Type", "operator": "EQUAL", "value": "INDIVIDUAL"}
        ],
        "outputFields": [
            "Account ID",
            "First Name",
            "Last Name",
            "Email 1",
            "Date Created",
        ],
        "pagination": {
            "currentPage": 0,
            "pageSize": 6,  # Small page size for demo
        },
    }

    try:
        print("🔎 Searching for individual accounts...")
        print("📋 Search criteria: Account Type = INDIVIDUAL")
        print(f"📄 Page size: {search_request['pagination']['pageSize']}")
        print()

        count = 0
        page_count = 0

        # The search method handles pagination automatically too!
        for account in client.accounts.search(search_request):
            count += 1

            # Track page boundaries for demonstration
            if (count - 1) % search_request["pagination"]["pageSize"] == 0:
                page_count += 1
                print(f"📄 Page {page_count}:")

            account_id = account.get("Account ID")
            first_name = account.get("First Name", "")
            last_name = account.get("Last Name", "")
            email = account.get("Email 1", "No email")
            created = account.get("Date Created", "Unknown")

            page_item = (count - 1) % search_request["pagination"]["pageSize"] + 1
            print(f"   {page_item}. ID {account_id}: {first_name} {last_name}")
            print(f"      Email: {email}")
            print(f"      Created: {created}")
            print()

            # Limit results for demo
            if count >= 18:
                print(f"   ... (showing first {count} search results)")
                break

        print("✅ Search Pagination Summary:")
        print(f"   Total results processed: {count}")
        print(f"   Pages processed: {page_count}")
        print(f"   Results per page: {search_request['pagination']['pageSize']}")

        print("\n💡 Search Pagination Benefits:")
        print("   • Filter data before pagination (more efficient)")
        print("   • Control exactly which fields are returned")
        print("   • Combine multiple search criteria")
        print("   • Automatic pagination handling like list()")

    except Exception as e:
        print(f"❌ Error in search pagination: {e}")


# Run the example
search_pagination_example()

## Example 4: Pagination with Filtering

Combine pagination with additional filtering to process only the data you need.

In [ ]:
def pagination_with_filtering():
    """Pagination combined with filtering."""
    print("\n🔧 Pagination with Filtering")
    print("=" * 35)
    print("Combine API-level and application-level filtering.")
    print()

    try:
        # Example 1: API-level filtering (Individual accounts)
        print("👥 Individual accounts with email addresses:")
        individual_count = 0
        individual_with_email = 0

        for account in client.accounts.list(user_type=UserType.INDIVIDUAL, page_size=8):
            individual_count += 1

            # Application-level filtering - only show accounts with email
            email = account.get("email")
            if email:
                individual_with_email += 1
                first_name = account.get("firstName", "")
                last_name = account.get("lastName", "")

                print(
                    f"   {individual_with_email:2}. {first_name} {last_name} - {email}"
                )

                if individual_with_email >= 8:
                    print("   ... (showing first 8 with emails)")
                    break

            # Safety limit
            if individual_count >= 50:
                break

        print("\n📊 Individual Accounts Summary:")
        print(f"   Accounts processed: {individual_count}")
        print(f"   Accounts with email: {individual_with_email}")
        if individual_count > 0:
            print(
                f"   Email percentage: {(individual_with_email / individual_count) * 100:.1f}%"
            )

        # Example 2: Company accounts
        print("\n🏢 Company accounts:")
        company_count = 0

        for account in client.accounts.list(user_type=UserType.COMPANY, page_size=6):
            company_count += 1
            company_name = account.get("companyName", "Unknown Company")
            email = account.get("email", "No email")
            website = account.get("website", "No website")

            print(f"   {company_count}. {company_name}")
            print(f"      📧 {email}")
            print(f"      🌐 {website}")
            print()

            if company_count >= 5:
                print("   ... (showing first 5 companies)")
                break

        print("✅ Filtering Summary:")
        print(
            f"   Individual accounts: {individual_count} processed, {individual_with_email} with email"
        )
        print(f"   Company accounts: {company_count} processed")

        print("\n💡 Filtering Best Practices:")
        print("   • Use API filters (user_type) to reduce data transfer")
        print("   • Apply additional filters in your application logic")
        print("   • Consider using search() for complex filtering")
        print("   • Monitor API usage and response times")

    except Exception as e:
        print(f"❌ Error in filtered pagination: {e}")


# Run the example
pagination_with_filtering()

## Example 5: Bulk Processing with Pagination

Process large datasets efficiently using pagination strategies optimized for bulk operations.

In [ ]:
def bulk_processing_example():
    """Example of processing large datasets with optimized pagination."""
    print("\n📦 Bulk Processing with Pagination")
    print("=" * 40)
    print("Strategies for processing large datasets efficiently.")
    print()

    try:
        batch_size = 25  # Larger batches for efficiency
        total_accounts = 0
        accounts_with_email = 0
        accounts_without_email = 0
        accounts_with_phone = 0
        processing_start_time = time.time()

        print(f"🔄 Processing accounts in batches of {batch_size}...")
        print("📊 Tracking statistics as we go...")
        print()

        for account in client.accounts.list(
            page_size=batch_size, user_type=UserType.INDIVIDUAL
        ):
            total_accounts += 1

            # Collect statistics
            if account.get("email"):
                accounts_with_email += 1
            else:
                accounts_without_email += 1

            if account.get("phone"):
                accounts_with_phone += 1

            # Progress indicator every batch
            if total_accounts % batch_size == 0:
                elapsed = time.time() - processing_start_time
                rate = total_accounts / elapsed if elapsed > 0 else 0

                print(
                    f"   📈 Processed {total_accounts:,} accounts ({rate:.1f} accounts/sec)"
                )
                print(
                    f"      📧 With email: {accounts_with_email:,} ({(accounts_with_email / total_accounts) * 100:.1f}%)"
                )
                print(
                    f"      📞 With phone: {accounts_with_phone:,} ({(accounts_with_phone / total_accounts) * 100:.1f}%)"
                )
                print()

            # Stop for demo purposes (remove this in real bulk processing)
            if total_accounts >= 100:
                print(f"   🛑 Stopping at {total_accounts} accounts for demo")
                break

        processing_time = time.time() - processing_start_time

        print("✅ Bulk Processing Results:")
        print(f"   Total accounts processed: {total_accounts:,}")
        print(f"   Processing time: {processing_time:.2f} seconds")
        print(
            f"   Average rate: {total_accounts / processing_time:.1f} accounts/second"
        )
        print(f"   Batch size used: {batch_size}")
        print()

        print("📊 Contact Information Analysis:")
        print(
            f"   Accounts with email: {accounts_with_email:,} ({(accounts_with_email / total_accounts) * 100:.1f}%)"
        )
        print(
            f"   Accounts without email: {accounts_without_email:,} ({(accounts_without_email / total_accounts) * 100:.1f}%)"
        )
        print(
            f"   Accounts with phone: {accounts_with_phone:,} ({(accounts_with_phone / total_accounts) * 100:.1f}%)"
        )

        print("\n💡 Bulk Processing Tips:")
        print("   • Use larger page sizes (50-200) for better throughput")
        print("   • Process items immediately, don't collect into lists")
        print("   • Show progress indicators for long operations")
        print("   • Consider parallel processing for independent operations")
        print("   • Monitor memory usage and API rate limits")

    except Exception as e:
        print(f"❌ Error in bulk processing: {e}")


# Run the example
bulk_processing_example()

## Pagination Performance Tips and Best Practices

Learn the key principles for efficient pagination.

In [ ]:
def pagination_performance_tips():
    """Comprehensive guide to pagination performance and best practices."""
    print("\n⚡ Pagination Performance Tips & Best Practices")
    print("=" * 55)

    print("\n🎯 1. PAGE SIZE OPTIMIZATION")
    print("=" * 30)
    print("Choose the right page size for your use case:")
    print()

    page_size_guide = {
        "Interactive Display (UI)": "10-25 items",
        "Standard Processing": "50-100 items",
        "Bulk Operations": "100-500 items",
        "Data Export": "200-1000 items",
        "Memory-Constrained": "25-50 items",
    }

    for use_case, recommendation in page_size_guide.items():
        print(f"   📋 {use_case:25} → {recommendation}")

    print("\n⚠️  Too small: More API calls, slower overall")
    print("⚠️  Too large: Higher memory usage, slower individual requests")

    print("\n🧠 2. MEMORY MANAGEMENT")
    print("=" * 25)
    print("✅ EFFICIENT: Process items one at a time")
    print("```python")
    print("for account in client.accounts.list(page_size=100):")
    print("    process_account(account)  # Process immediately")
    print("```")
    print()
    print("❌ INEFFICIENT: Loading all into memory first")
    print("```python")
    print("# Don't do this for large datasets!")
    print("all_accounts = list(client.accounts.list())")
    print("for account in all_accounts:")
    print("    process_account(account)")
    print("```")

    print("\n🎯 3. API EFFICIENCY")
    print("=" * 20)
    efficiency_tips = [
        "Use search() with outputFields to get only needed data",
        "Apply filters at API level, not in Python",
        "Cache field metadata to avoid repeated lookups",
        "Use appropriate user_type filters",
        "Consider date range filters for time-based data",
    ]

    for i, tip in enumerate(efficiency_tips, 1):
        print(f"   {i}. {tip}")

    print("\n🛡️ 4. ERROR HANDLING & RESILIENCE")
    print("=" * 35)
    resilience_tips = [
        "Implement retry logic for transient network errors",
        "Handle rate limiting gracefully (exponential backoff)",
        "Save progress periodically for long-running processes",
        "Log errors with context (page number, account ID, etc.)",
        "Implement checkpointing for resumable operations",
    ]

    for i, tip in enumerate(resilience_tips, 1):
        print(f"   {i}. {tip}")

    print("\n📈 5. PERFORMANCE MONITORING")
    print("=" * 30)
    print("Track these metrics:")
    monitoring_metrics = [
        "Items processed per second",
        "API response times",
        "Memory usage over time",
        "Network bandwidth utilization",
        "Error rates and retry counts",
    ]

    for metric in monitoring_metrics:
        print(f"   📊 {metric}")

    print("\n🔧 6. ADVANCED PATTERNS")
    print("=" * 25)
    print("✅ Using search with specific fields:")
    print("```python")
    print("search_request = {")
    print("    'outputFields': ['Account ID', 'First Name', 'Email 1'],")
    print("    'pagination': {'pageSize': 100},")
    print(
        "    'searchFields': [{'field': 'Account Type', 'operator': 'EQUAL', 'value': 'Individual'}]"
    )
    print("}")
    print("for account in client.accounts.search(search_request):")
    print("    process_account(account)")
    print("```")

    print("\n✅ Progress tracking for long operations:")
    print("```python")
    print("processed = 0")
    print("for account in client.accounts.list(page_size=100):")
    print("    process_account(account)")
    print("    processed += 1")
    print("    if processed % 1000 == 0:")
    print("        print(f'Processed {processed:,} accounts...')")
    print("```")

    print("\n🎯 7. COMMON PITFALLS TO AVOID")
    print("=" * 35)
    pitfalls = [
        "Converting iterators to lists with list() - causes memory issues",
        "Using very small page sizes (< 10) - inefficient API usage",
        "Using very large page sizes (> 1000) - memory and timeout issues",
        "Not handling API errors - operations fail silently",
        "Ignoring rate limits - gets your API access throttled",
        "Processing data in nested loops - O(n²) performance",
    ]

    for i, pitfall in enumerate(pitfalls, 1):
        print(f"   ❌ {i}. {pitfall}")


# Show the comprehensive guide
pagination_performance_tips()

## Example 6: Real-World Pagination Scenarios

Practical examples of common pagination use cases.

In [ ]:
def real_world_scenarios():
    """Demonstrate common real-world pagination scenarios."""
    print("\n🌍 Real-World Pagination Scenarios")
    print("=" * 40)

    # Scenario 1: Data Export
    print("📊 Scenario 1: Data Export for Reporting")
    print("-" * 40)
    print("Exporting contact data with specific fields for external reporting.")
    print()

    try:
        export_data = []
        export_count = 0

        print("🔄 Exporting contact data...")

        # Use search for precise field control
        export_search = {
            "searchFields": [
                {"field": "Account Type", "operator": "EQUAL", "value": "INDIVIDUAL"}
            ],
            "outputFields": [
                "Account ID",
                "First Name",
                "Last Name",
                "Email 1",
                "Phone 1",
                "Date Created",
            ],
            "pagination": {"currentPage": 0, "pageSize": 50},  # Larger pages for export
        }

        for account in client.accounts.search(export_search):
            # Create export record
            export_record = {
                "id": account.get("Account ID"),
                "name": f"{account.get('First Name', '')} {account.get('Last Name', '')}".strip(),
                "email": account.get("Email 1", ""),
                "phone": account.get("Phone 1", ""),
                "created": account.get("Date Created", ""),
            }

            export_data.append(export_record)
            export_count += 1

            # Show progress
            if export_count % 25 == 0:
                print(f"   📈 Exported {export_count} records...")

            # Limit for demo
            if export_count >= 50:
                break

        print(f"✅ Export completed: {len(export_data)} records")
        print(f"   First record: {export_data[0] if export_data else 'None'}")
        print()

    except Exception as e:
        print(f"❌ Export error: {e}")
        print()

    # Scenario 2: Data Validation
    print("🔍 Scenario 2: Data Quality Validation")
    print("-" * 40)
    print("Scanning accounts to identify data quality issues.")
    print()

    try:
        validation_stats = {
            "total_checked": 0,
            "missing_email": 0,
            "missing_phone": 0,
            "incomplete_name": 0,
            "suspicious_email": 0,
        }

        print("🔍 Validating account data quality...")

        for account in client.accounts.list(
            page_size=30, user_type=UserType.INDIVIDUAL
        ):
            validation_stats["total_checked"] += 1

            # Check for missing data
            if not account.get("email"):
                validation_stats["missing_email"] += 1

            if not account.get("phone"):
                validation_stats["missing_phone"] += 1

            # Check name completeness
            first_name = account.get("firstName", "").strip()
            last_name = account.get("lastName", "").strip()
            if not first_name or not last_name:
                validation_stats["incomplete_name"] += 1

            # Check email format (basic)
            email = account.get("email", "")
            if email and "@" not in email:
                validation_stats["suspicious_email"] += 1

            # Progress indicator
            if validation_stats["total_checked"] % 25 == 0:
                print(
                    f"   📊 Validated {validation_stats['total_checked']} accounts..."
                )

            # Limit for demo
            if validation_stats["total_checked"] >= 75:
                break

        print("\n📊 Data Quality Report:")
        total = validation_stats["total_checked"]
        if total > 0:
            print(f"   Total accounts checked: {total}")
            print(
                f"   Missing email: {validation_stats['missing_email']} ({validation_stats['missing_email'] / total * 100:.1f}%)"
            )
            print(
                f"   Missing phone: {validation_stats['missing_phone']} ({validation_stats['missing_phone'] / total * 100:.1f}%)"
            )
            print(
                f"   Incomplete names: {validation_stats['incomplete_name']} ({validation_stats['incomplete_name'] / total * 100:.1f}%)"
            )
            print(
                f"   Suspicious emails: {validation_stats['suspicious_email']} ({validation_stats['suspicious_email'] / total * 100:.1f}%)"
            )
        print()

    except Exception as e:
        print(f"❌ Validation error: {e}")
        print()

    # Scenario 3: Selective Processing
    print("🎯 Scenario 3: Selective Processing by Criteria")
    print("-" * 45)
    print("Processing only accounts that meet specific business criteria.")
    print()

    try:
        processing_stats = {
            "total_scanned": 0,
            "email_subscribers": 0,
            "phone_contacts": 0,
            "complete_profiles": 0,
        }

        print("🔄 Identifying high-value contacts...")

        for account in client.accounts.list(
            page_size=40, user_type=UserType.INDIVIDUAL
        ):
            processing_stats["total_scanned"] += 1

            email = account.get("email", "")
            phone = account.get("phone", "")
            first_name = account.get("firstName", "").strip()
            last_name = account.get("lastName", "").strip()

            # Business criteria
            has_email = bool(email and "@" in email)
            has_phone = bool(phone)
            has_complete_name = bool(first_name and last_name)

            if has_email:
                processing_stats["email_subscribers"] += 1

            if has_phone:
                processing_stats["phone_contacts"] += 1

            if has_email and has_phone and has_complete_name:
                processing_stats["complete_profiles"] += 1
                # This would be where you'd process high-value contacts
                # e.g., add to special campaign, update priority score, etc.

            if processing_stats["total_scanned"] % 30 == 0:
                print(
                    f"   📈 Processed {processing_stats['total_scanned']} accounts..."
                )

            # Limit for demo
            if processing_stats["total_scanned"] >= 90:
                break

        print("\n🎯 Selective Processing Results:")
        total = processing_stats["total_scanned"]
        if total > 0:
            print(f"   Total accounts scanned: {total}")
            print(
                f"   Email subscribers: {processing_stats['email_subscribers']} ({processing_stats['email_subscribers'] / total * 100:.1f}%)"
            )
            print(
                f"   Phone contacts: {processing_stats['phone_contacts']} ({processing_stats['phone_contacts'] / total * 100:.1f}%)"
            )
            print(
                f"   Complete profiles: {processing_stats['complete_profiles']} ({processing_stats['complete_profiles'] / total * 100:.1f}%)"
            )

        print("\n💡 Real-World Applications:")
        print("   • Data exports for business intelligence")
        print("   • Data quality audits and cleanup")
        print("   • Segmentation for targeted campaigns")
        print("   • Migration validation and verification")
        print("   • Automated data enrichment processes")

    except Exception as e:
        print(f"❌ Processing error: {e}")


# Run the real-world scenarios
real_world_scenarios()

## Cleanup

Don't forget to close the client connection when you're done:

In [ ]:
# Close the client connection
client.close()
print("✅ Client connection closed.")

## Summary: Mastering Pagination

### Key Concepts Learned

1. **Automatic Pagination**: The SDK handles pagination seamlessly with `.list()` and `.search()`
2. **Manual Control**: Use specific page parameters when you need fine-grained control
3. **Search Pagination**: Combine filtering and pagination for efficient data retrieval
4. **Performance Optimization**: Choose appropriate page sizes and processing patterns
5. **Real-World Applications**: Export, validation, and selective processing scenarios

### Best Practices Summary

#### Page Size Guidelines
- **Interactive UI**: 10-25 items
- **Standard Processing**: 50-100 items
- **Bulk Operations**: 100-500 items
- **Data Export**: 200-1000 items

#### Memory Management
- ✅ Process items one at a time (streaming)
- ❌ Avoid loading entire datasets into memory
- ✅ Use generators and iterators
- ❌ Don't convert iterators to lists unnecessarily

#### Performance Optimization
- Use `search()` with `outputFields` to get only needed data
- Apply filters at the API level, not in application code
- Monitor processing rates and adjust page sizes accordingly
- Implement progress indicators for long-running operations

#### Error Handling
- Implement retry logic for transient errors
- Handle rate limiting gracefully
- Save progress for resumable operations
- Log errors with sufficient context

### Common Patterns

```python
# ✅ Efficient: Automatic pagination with processing
for account in client.accounts.list(page_size=100):
    process_account(account)

# ✅ Efficient: Search with specific fields
search_request = {
    "outputFields": ["Account ID", "First Name", "Email 1"],
    "pagination": {"pageSize": 100}
}
for account in client.accounts.search(search_request):
    process_account(account)

# ✅ Efficient: Progress tracking
processed = 0
for account in client.accounts.list(page_size=100):
    process_account(account)
    processed += 1
    if processed % 1000 == 0:
        print(f"Processed {processed:,} accounts...")
```

### When to Use Each Approach

- **Automatic Pagination**: Most common use case, when you need to process all matching records
- **Manual Pagination**: When you need to process specific page ranges or implement custom progress tracking
- **Search Pagination**: When you need complex filtering or specific output fields
- **Bulk Processing**: For data exports, migrations, or analysis of large datasets

### Next Steps

Now that you've mastered pagination, you can:

- **Scale Your Applications**: Handle datasets of any size efficiently
- **Optimize Performance**: Choose the right pagination strategy for each use case
- **Build Robust Systems**: Implement proper error handling and progress tracking
- **Create Reports**: Export and analyze your CRM data effectively

Pagination is a fundamental skill for working with any API - these patterns will serve you well beyond just the Neon CRM SDK! 🚀